In [2]:
# ==========================================
# HDW HOURS CALCULATION FROM ERA5 ZIP FILES
# Using phenology DOY logic
# ==========================================

!pip install xarray netCDF4 pandas numpy tqdm

import os
import zipfile
import numpy as np
import pandas as pd
import xarray as xr
from tqdm import tqdm
from datetime import datetime, timedelta

# ------------------------------------------
# Mount Google Drive
# ------------------------------------------
from google.colab import drive
drive.mount('/content/drive')

# ------------------------------------------
# Paths
# ------------------------------------------
ERA5_DIR = "/content/drive/MyDrive/ERA5"   # ERA5 zip folder
LOC_FILE = "/content/drive/MyDrive/ERA5/Locations.csv"  # Your locations CSV
OUTPUT_FILE = "/content/drive/MyDrive/HDW_Stages.csv"

# ------------------------------------------
# Helper functions
# ------------------------------------------
def doy_to_date(year, doy):
    """Convert year + DOY to a datetime.date object."""
    return datetime(year, 1, 1) + timedelta(days=int(doy) - 1)

def rh_from_temp_dew(temp_c, dew_c):
    """Calculate RH from temperature and dewpoint."""
    es = 6.112 * np.exp((17.67 * temp_c) / (temp_c + 243.5))
    e = 6.112 * np.exp((17.67 * dew_c) / (dew_c + 243.5))
    return (e / es) * 100.0

def unzip_and_load(year, month):
    """Unzip and load ERA5 NetCDF for given year/month."""
    zip_name = f"era5_egypt_{year:04d}_{month:02d}.zip"
    zip_path = os.path.join(ERA5_DIR, zip_name)
    if not os.path.exists(zip_path):
        return None

    tmp_dir = "/content/tmp_nc"
    os.makedirs(tmp_dir, exist_ok=True)

    with zipfile.ZipFile(zip_path, 'r') as z:
        nc_files = [f for f in z.namelist() if f.endswith(".nc")]
        if len(nc_files) == 0:
            return None
        z.extract(nc_files[0], tmp_dir)
        nc_path = os.path.join(tmp_dir, nc_files[0])

    try:
        ds = xr.open_dataset(nc_path)
        return ds
    except:
        return None

def extract_hdw_hours(lat, lon, start_date, end_date):
    """Count HDW hours between two dates."""
    months = pd.date_range(start=start_date, end=end_date, freq='MS')
    hdw_count = 0

    for m in months:
        ds = unzip_and_load(m.year, m.month)
        if ds is None:
            continue

        # Select nearest point
        ds_point = ds.sel(latitude=lat, longitude=lon, method="nearest")

        # Variables
        t2m = ds_point['t2m'] - 273.15  # °C
        d2m = ds_point['d2m'] - 273.15  # °C
        u10 = ds_point['u10']
        v10 = ds_point['v10']
        wspd = np.sqrt(u10**2 + v10**2)
        rh = rh_from_temp_dew(t2m, d2m)

        df = pd.DataFrame({
            "time": pd.to_datetime(t2m.time.values),
            "T": t2m.values,
            "RH": rh,
            "WSPD": wspd.values
        })

        df = df[(df["time"] >= start_date) & (df["time"] <= end_date)]
        hdw_hours = df[(df["T"] >= 32) & (df["RH"] <= 30) & (df["WSPD"] >= 7)]
        hdw_count += len(hdw_hours)

    return hdw_count

# ------------------------------------------
# Load and prepare locations
# ------------------------------------------
locs = pd.read_csv(LOC_FILE)

results = []

for _, row in tqdm(locs.iterrows(), total=len(locs)):
    loc = row["Location"]
    year = int(row["Years"])
    lat = row["Latitude"]
    lon = row["Longitude"]

    # Compute correct stage dates from DOY rules
    sow_date = doy_to_date(year - 1, 313)  # always in previous year
    anth_date = doy_to_date(year, row["ADAT(DOY)"])
    mat_date = doy_to_date(year, row["MDAT(DOY)"])

    # Stage 1: Sowing → Anthesis (crosses year boundary)
    hdw_sow_anth = extract_hdw_hours(lat, lon, sow_date, anth_date)

    # Stage 2: Sowing → Maturity (crosses year boundary)
    hdw_sow_mat = extract_hdw_hours(lat, lon, sow_date, mat_date)

    # Stage 3: Anthesis → Maturity (same year)
    hdw_anth_mat = extract_hdw_hours(lat, lon, anth_date, mat_date)

    results.append({
        "Location": loc,
        "Year": year,
        "HDW_SowToAnth": hdw_sow_anth,
        "HDW_SowToMat": hdw_sow_mat,
        "HDW_AnthToMat": hdw_anth_mat
    })

# ------------------------------------------
# Save results
# ------------------------------------------
df_out = pd.DataFrame(results)
df_out.to_csv(OUTPUT_FILE, index=False)
print(f"Saved results to: {OUTPUT_FILE}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


  0%|          | 0/1920 [00:02<?, ?it/s]


AttributeError: 'DataArray' object has no attribute 'time'

In [3]:
# =========================================
# 1. Install and Import Libraries
# =========================================
!pip install xarray netCDF4 pandas numpy tqdm

import xarray as xr
import pandas as pd
import numpy as np
import zipfile
import os
from tqdm import tqdm
from google.colab import drive

# =========================================
# 2. Mount Google Drive
# =========================================
drive.mount('/content/drive')

# Path to ERA5 zipped files and Locations.csv in your Google Drive
ERA5_DIR = "/content/drive/MyDrive/ERA5"  # Change if needed
LOC_FILE = "/content/drive/MyDrive/ERA5/Locations.csv"  # Change if needed
OUTPUT_FILE = "/content/drive/MyDrive/HDW_Stages.csv"

# =========================================
# 3. Helper: Convert DOY to Date
# =========================================
def doy_to_date(year, doy):
    """Convert Day of Year to datetime.date"""
    return pd.Timestamp(year=year, month=1, day=1) + pd.Timedelta(days=doy - 1)

# =========================================
# 4. Load Locations and Fix Dates
# =========================================
locs = pd.read_csv(LOC_FILE)

# Adjust for cross-year sowing logic
locs["SowingDate"] = [doy_to_date(y-1, 313) for y in locs["Years"]]
locs["AnthesisDate"] = [doy_to_date(y, adoy) for y, adoy in zip(locs["Years"], locs["ADAT(DOY)"])]
locs["MaturityDate"] = [doy_to_date(y, mdoy) for y, mdoy in zip(locs["Years"], locs["MDAT(DOY)"])]

# =========================================
# 5. Helper: Unzip and Load ERA5 NetCDF
# =========================================
def unzip_and_load(year, month):
    """Unzip ERA5 monthly file and load with xarray"""
    zip_name = f"era5_egypt_{year:04d}_{month:02d}.zip"
    zip_path = os.path.join(ERA5_DIR, zip_name)
    if not os.path.exists(zip_path):
        return None

    with zipfile.ZipFile(zip_path, 'r') as z:
        nc_files = [f for f in z.namelist() if f.endswith(".nc")]
        if not nc_files:
            return None
        z.extract(nc_files[0], "/content")
        nc_path = os.path.join("/content", nc_files[0])
        ds = xr.open_dataset(nc_path)
    return ds

# =========================================
# 6. Helper: Compute RH from T and Dew Point
# =========================================
def rh_from_temp_dew(t, td):
    """Calculate relative humidity from T and dew point (°C)"""
    es = 6.112 * np.exp((17.67 * t) / (t + 243.5))
    e = 6.112 * np.exp((17.67 * td) / (td + 243.5))
    return (e / es) * 100

# =========================================
# 7. Extract HDW Hours for a Given Period
# =========================================
def extract_hdw_hours(lat, lon, start_date, end_date):
    """Count HDW hours between two dates"""
    months = pd.date_range(start=start_date, end=end_date, freq='MS')
    hdw_count = 0

    for m in months:
        ds = unzip_and_load(m.year, m.month)
        if ds is None:
            continue

        # Select nearest point and convert to proper units
        t2m = ds['t2m'].sel(latitude=lat, longitude=lon, method="nearest") - 273.15
        d2m = ds['d2m'].sel(latitude=lat, longitude=lon, method="nearest") - 273.15
        u10 = ds['u10'].sel(latitude=lat, longitude=lon, method="nearest")
        v10 = ds['v10'].sel(latitude=lat, longitude=lon, method="nearest")

        # Get time from coordinates
        times = pd.to_datetime(t2m.coords['time'].values)

        # Wind speed
        wspd = np.sqrt(u10**2 + v10**2)

        # Relative humidity
        rh = rh_from_temp_dew(t2m.values, d2m.values)

        # Make dataframe
        df = pd.DataFrame({
            "time": times,
            "T": t2m.values,
            "RH": rh,
            "WSPD": wspd.values
        })

        # Filter to date range
        df = df[(df["time"] >= pd.Timestamp(start_date)) & (df["time"] <= pd.Timestamp(end_date))]

        # Apply HDW condition
        hdw_hours = df[(df["T"] >= 32) & (df["RH"] <= 30) & (df["WSPD"] >= 7)]
        hdw_count += len(hdw_hours)

    return hdw_count

# =========================================
# 8. Main Loop for All Locations and Years
# =========================================
results = []

for idx, row in tqdm(locs.iterrows(), total=len(locs)):
    lat = row["Latitude"]
    lon = row["Longitude"]

    sow_date = row["SowingDate"]
    anth_date = row["AnthesisDate"]
    mat_date = row["MaturityDate"]

    # Stage 1: Sowing → Anthesis
    hdw_sow_anth = extract_hdw_hours(lat, lon, sow_date, anth_date)

    # Stage 2: Sowing → Maturity
    hdw_sow_mat = extract_hdw_hours(lat, lon, sow_date, mat_date)

    # Stage 3: Anthesis → Maturity
    hdw_anth_mat = extract_hdw_hours(lat, lon, anth_date, mat_date)

    results.append({
        "Location": row["Location"],
        "Year": row["Years"],
        "HDW_SowToAnth": hdw_sow_anth,
        "HDW_SowToMat": hdw_sow_mat,
        "HDW_AnthToMat": hdw_anth_mat
    })

# =========================================
# 9. Save Results
# =========================================
df_results = pd.DataFrame(results)
df_results.to_csv(OUTPUT_FILE, index=False)
print(f"✅ HDW results saved to {OUTPUT_FILE}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


  0%|          | 0/1920 [00:00<?, ?it/s]


KeyError: 'time'

In [4]:
# =============================================
# 1. Install & Import Libraries
# =============================================
!pip install xarray netCDF4 pandas numpy tqdm --quiet

import xarray as xr
import pandas as pd
import numpy as np
import zipfile
import os
from tqdm import tqdm

# =============================================
# 2. Set Paths
# =============================================
# Adjust to your Google Drive mount path
era5_dir = "/content/drive/MyDrive/ERA5"  # Folder with zipped ERA5 monthly files
locations_file = "/content/drive/MyDrive/ERA5/Locations.csv"
output_file = "/content/drive/MyDrive/HDW_PhenoStages.csv"

# =============================================
# 3. Load Locations and Fix Dates
# =============================================
locs = pd.read_csv(locations_file)

# Helper: Convert DOY → Date
def doy_to_date(year, doy):
    return pd.to_datetime(year.astype(str) + '-01-01') + pd.to_timedelta(doy - 1, unit='D')

# Cross-year sowing logic
locs["SowingDate"] = doy_to_date(locs["Years"] - 1, locs["SDAT(DOY)"])
locs["AnthesisDate"] = doy_to_date(locs["Years"], locs["ADAT(DOY)"])
locs["MaturityDate"] = doy_to_date(locs["Years"], locs["MDAT(DOY)"])

# =============================================
# 4. Function to Load ERA5 .nc from Zip
# =============================================
def load_nc_from_zip(year, month, varname="t2m"):
    zip_name = f"era5_egypt_{year:04d}_{month:02d}.zip"
    zip_path = os.path.join(era5_dir, zip_name)

    if not os.path.exists(zip_path):
        return None

    with zipfile.ZipFile(zip_path, 'r') as z:
        nc_files = [f for f in z.namelist() if f.endswith(".nc")]
        if not nc_files:
            return None
        nc_filename = nc_files[0]
        z.extract(nc_filename, "/tmp")
        nc_path = os.path.join("/tmp", nc_filename)
        ds = xr.open_dataset(nc_path)

    # Detect correct time coordinate
    possible_time_vars = ["time", "valid_time", "date", "forecast_time"]
    time_var = None
    for tv in possible_time_vars:
        if tv in ds.coords:
            time_var = tv
            break
    if time_var is None:
        raise ValueError(f"No time coordinate found in dataset: {list(ds.coords)}")

    # Convert Kelvin → Celsius if t2m
    if varname in ds:
        if ds[varname].max() > 200:  # crude check for Kelvin
            ds[varname] = ds[varname] - 273.15

    return ds, varname, time_var

# =============================================
# 5. Function to Calculate HDW Hours
# =============================================
def extract_hdw_hours(lat, lon, start_date, end_date):
    if pd.isna(start_date) or pd.isna(end_date):
        return np.nan

    months_needed = pd.date_range(start_date, end_date, freq='MS')

    all_hdw_hours = []
    for m in months_needed:
        loaded = load_nc_from_zip(m.year, m.month, varname="t2m")
        if loaded is None:
            continue
        ds, varname, time_var = loaded

        # Find nearest grid point
        ds_point = ds[varname].sel(longitude=lon, latitude=lat, method="nearest")

        times = pd.to_datetime(ds_point[time_var].values)
        mask = (times >= pd.to_datetime(start_date)) & (times <= pd.to_datetime(end_date))
        vals = ds_point.values[mask]

        if len(vals) > 0:
            # Example HDW condition: Tmax > 30°C (you can adjust criteria)
            hdw_hours = np.sum(vals > 30)
            all_hdw_hours.append(hdw_hours)

        ds.close()

    if not all_hdw_hours:
        return np.nan
    return np.sum(all_hdw_hours)

# =============================================
# 6. Main Loop for All Locations
# =============================================
results = []
for idx, row in tqdm(locs.iterrows(), total=len(locs)):
    lat = row["Latitude"]
    lon = row["Longitude"]

    sow_date = row["SowingDate"]
    anth_date = row["AnthesisDate"]
    mat_date = row["MaturityDate"]

    # Stage 1: Sowing → Anthesis
    hdw_sow_anth = extract_hdw_hours(lat, lon, sow_date, anth_date)

    # Stage 2: Sowing → Maturity
    hdw_sow_mat = extract_hdw_hours(lat, lon, sow_date, mat_date)

    # Stage 3: Anthesis → Maturity
    hdw_anth_mat = extract_hdw_hours(lat, lon, anth_date, mat_date)

    results.append({
        "Location": row["Location"],
        "Year": row["Years"],
        "HDW_SowToAnth": hdw_sow_anth,
        "HDW_SowToMat": hdw_sow_mat,
        "HDW_AnthToMat": hdw_anth_mat
    })

# =============================================
# 7. Save Results
# =============================================
results_df = pd.DataFrame(results)
results_df.to_csv(output_file, index=False)
print(f"Saved results to {output_file}")


100%|██████████| 1920/1920 [1:01:38<00:00,  1.93s/it]

Saved results to /content/drive/MyDrive/HDW_PhenoStages.csv
